In [ ]:
library(dplyr)
library(Matrix)
library(Seurat)
library(ggplot2)
library(sctransform)
library(reticulate)
library(ggplot2)
library(patchwork)
library(devtools) 
library(png)
library(SeuratDisk)
library(Nebulosa)
library("AnnotationDbi")
library("org.Hs.eg.db")
library("clusterProfiler")
library(limma)
set.seed(1234)


# Read in combined data

In [ ]:
se.big.singlet2 <- readRDS('./scrna_lecMicro_preprocess_df.RDS')

In [ ]:
# subset FIRE mice - of interest in this experiment
se.big.singlet2 = subset(se.big.singlet2, background == "FIRE")

se.big.singlet2@meta.data$Treatment <- ifelse(se.big.singlet2@meta.data$hash.ID %in% c(
    "AB01", "AB02", "AB05", "AB13", "AB14", "AB40", 
    "AB57", "AB41", "AB49", "AB45", "AB92", "AB93", "AB88"), 
    "Lec", "iGg1")

# STransform and processing of QC'ed dataset

In [ ]:
DefaultAssay(se.big.singlet2) <- "RNA"

micro.list <- SplitObject(se.big.singlet2, split.by = "Library")

micro.list <- lapply(X = micro.list, FUN = SCTransform)
features <- SelectIntegrationFeatures(object.list = micro.list, nfeatures = 3000)

micro.list <- PrepSCTIntegration(object.list = micro.list, anchor.features = features)

anchors <- FindIntegrationAnchors(object.list = micro.list, normalization.method = "SCT",
    anchor.features = features)

combined.sct <- IntegrateData(anchorset = anchors, normalization.method = "SCT")

combined.sct <- RunPCA(combined.sct, verbose = FALSE)


In [ ]:
DefaultAssay(combined.sct) <- "integrated"

combined.sct <- RunUMAP(combined.sct, reduction = "pca", dims = 1:30)

micro.combined3 = combined.sct
micro.combined3 <- FindNeighbors(micro.combined3, reduction = "pca", dims = 1:30,k.param = 100, nn.method = "annoy", annoy.metric = "cosine")


In [ ]:
DefaultAssay(micro.combined3) <- "integrated"
micro.combined3 <- FindClusters(micro.combined3, resolution = 0.3, n.iter=100)

options(repr.plot.width=12, repr.plot.height=10)
DimPlot(micro.combined3, reduction = "umap",  label = T, pt.size=1.5, label.size=8,raster=T) + 
    theme_classic(base_size=28)+ 
    guides(color=guide_legend(title="Cluster", override.aes = list(size=6)))

In [ ]:
options(repr.plot.width=22, repr.plot.height=10)

DimPlot(micro.combined3, reduction = "umap",  label = T, pt.size=1, ncol = 3,
        label.size=10,raster=F, split.by = "Library") + 
        theme_void(base_size=35)+ 
    guides(color=guide_legend(title="Cluster", override.aes = list(size=6)))

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

#visualize density of treatments 
micro.combined3@meta.data$Lec <- ifelse(micro.combined3@meta.data$Treatment
                                        == "Lec", 1/nrow(subset(micro.combined3@meta.data, Treatment == "Lec")), 0)

plot_density(micro.combined3, "Lec", size=2) + ggtitle("Lec")+ theme_void(base_size=19) + 
                     theme( plot.title = element_text(color="black", size=18, face="bold",  hjust = 0.5), 
                          legend.position = "none")  

In [ ]:
micro.combined3@meta.data$iGg1 <- ifelse(micro.combined3@meta.data$Treatment
                                        == "iGg1", 1/nrow(subset(micro.combined3@meta.data, Treatment == "iGg1")), 0)

plot_density(micro.combined3, "iGg1", size=2) + ggtitle("iGg1")+ theme_void(base_size=19) + 
                     theme( plot.title = element_text(color="black", size=18, face="bold",  hjust = 0.5), 
                          legend.position = "none")  

In [ ]:
# #remove human label from gene names 

# hu.genes.names = gsub('GRCh38-', '', se.big.singlet_test@assays$RNA@counts@Dimnames[[1]])
# se.big.singlet_test = se.big.singlet_test
# se.big.singlet_test@assays$RNA@counts@Dimnames[[1]]<-hu.genes.names
# se.big.singlet_test@assays$RNA@data@Dimnames[[1]] <- hu.genes.names
# df = as.data.frame(se.big.singlet_test@assays$RNA@counts@Dimnames[[1]])
# row.names(df) = df[[1]]
# x = subset(df, select = -1)
# se.big.singlet_test@assays$RNA@meta.features <- x

# hu.genes.names = gsub('GRCh38-', '', se.big.singlet_test@assays$RNA@data@Dimnames[[1]])
# se.big.singlet_test = se.big.singlet_test
# se.big.singlet_test@assays$RNA@data@Dimnames[[1]]<-hu.genes.names
# se.big.singlet_test@assays$RNA@data@Dimnames[[1]] <- hu.genes.names
# df = as.data.frame(se.big.singlet_test@assays$RNA@data@Dimnames[[1]])
# row.names(df) = df[[1]]
# x = subset(df, select = -1)
# se.big.singlet_test@assays$RNA@meta.features <- x


# hu.genes.names = gsub('GRCh38-', '', se.big.singlet_test@assays$SCT@data@Dimnames[[1]])
# se.big.singlet_test = se.big.singlet_test
# se.big.singlet_test@assays$SCT@data@Dimnames[[1]]<-hu.genes.names
# se.big.singlet_test@assays$SCT@data@Dimnames[[1]] <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', se.big.singlet_test@assays$SCT@counts@Dimnames[[1]])
# se.big.singlet_test = se.big.singlet_test
# se.big.singlet_test@assays$SCT@counts@Dimnames[[1]]<-hu.genes.names
# se.big.singlet_test@assays$SCT@counts@Dimnames[[1]] <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@scale.data))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@scale.data)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@scale.data) <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@SCTModel.list$model1@feature.attributes))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$model1@feature.attributes)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$model1@feature.attributes) <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.1`@feature.attributes))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.1`@feature.attributes)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.1`@feature.attributes) <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.2`@feature.attributes))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.2`@feature.attributes)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.2`@feature.attributes) <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.3`@feature.attributes))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.3`@feature.attributes)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.3`@feature.attributes) <- hu.genes.names


# hu.genes.names = gsub('GRCh38-', '', row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.4`@feature.attributes))
# se.big.singlet_test = se.big.singlet_test
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.4`@feature.attributes)<-hu.genes.names
# row.names(se.big.singlet_test@assays$SCT@SCTModel.list$`model1.4`@feature.attributes) <- hu.genes.names



In [ ]:
# find cluster markers

DefaultAssay(micro.combined3) <- 'SCT'

micro.combined3 = PrepSCTFindMarkers(micro.combined3, assay = "SCT", verbose = TRUE)

se.big.markers3_SCT <- FindAllMarkers(micro.combined3, logfc.threshold = 0.1, min.pct = 0.2, assay = "SCT", only.pos = TRUE)


In [ ]:
library(dittoSeq)
library(viridis)

se.big.markers.sig <- subset(se.big.markers3_SCT, p_val_adj < 0.05)

DefaultAssay(micro.combined3) <- 'RNA'

se.big.markers.top6 <-se.big.markers.sig %>%
    group_by(cluster) %>%
    slice_max(n = 6, order_by = -p_val_adj) %>%
    slice_max(n = 6, order_by = avg_log2FC)


options(repr.plot.width=29, repr.plot.height=9, repr.res = 300)

DotPlot(micro.combined3, features = unique(se.big.markers.top6$gene), 
        group.by = "integrated_snn_res.0.3", 
       col.min =-1, col.max = 2.2, dot.min = 0.05) + 
  geom_point(aes(size=pct.exp), shape = 21) +
  scale_colour_viridis(option="inferno", direction = -1) +
  guides(size=guide_legend(override.aes=list(shape=21 , fill="white")), 
        fill = guide_legend(title="Avg. Expr.") )+ 
  scale_size(range = c(1,10)) +
  theme_classic(base_size=25) +theme(
      axis.text.x = element_text(size = 20, angle = 90, vjust = 0.7, hjust=0.95),
      axis.text.y = element_text(size = 30)) + ylab("") + xlab("") +
  labs(color = "legend title") 



In [ ]:
DefaultAssay(micro.combined3) <- 'SCT'

# Define gene sets
micro_states <- list(
  DAM = "../../geneSets/mancuso032022/DAM.grp",
  HM = "../../geneSets/mancuso032022/HM.grp",
  IRM = "../../geneSets/mancuso032022/IRM.grp",
  TRM = "../../geneSets/mancuso032022/TRM.grp",
  CRM1 = "../../geneSets/mancuso032022/CRM1.grp",
  CRM2 = "../../geneSets/mancuso032022/CRM2.grp",
  HLA = "../../geneSets/mancuso032022/HLA.grp",
  tCRM = "../../geneSets/mancuso032022/tCRM.grp"
)

micro_states <- lapply(gene_sets, function(path) read.csv(path, header = FALSE)$V1)

# Add module scores
for (name in names(micro_states)) {
  micro.combined3 <- AddModuleScore(micro.combined3, features = list(micro_states[[name]]), name = paste0(name, "_enriched"))
}

signatures <- paste0(names(micro_states), "_enriched1")

options(repr.plot.width = 15, repr.plot.height = 10)
library(RColorBrewer)
library(gridExtra)

plotlist <- lapply(signatures, function(feature) {
  FeaturePlot(micro.combined3, features = feature, min.cutoff = 'q1', max.cutoff = 'q99',
              reduction = "umap", label = FALSE, pt.size = 0.7, label.size = 8, repel = TRUE) +
    scale_colour_gradientn(colours = rev(brewer.pal(n = 11, name = "Spectral"))) +  
    theme_void() + 
    theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"))
})

grid.arrange(grobs = plotlist, ncol = 3)


In [ ]:
saveRDS(micro.combined3, 'scrna_lecMicro_prefilter_sctransform.RDS')

# Remove macrophages and reprocess

In [ ]:
micro.combined4 = subset(micro.combined3, subset = integrated_snn_res.0.3 %in% c(0,1,2,3,4,5,6,7,8,10))

In [ ]:

DefaultAssay(micro.combined4) <- "RNA"

hu.genes.names = gsub('GRCh38-', '', micro.combined4@assays$RNA@counts@Dimnames[[1]])
micro.combined4@assays$RNA@counts@Dimnames[[1]]<-hu.genes.names
micro.combined4@assays$RNA@data@Dimnames[[1]] <- hu.genes.names
df = as.data.frame(micro.combined4@assays$RNA@counts@Dimnames[[1]])
row.names(df) = df[[1]]
x = subset(df, select = -1)
micro.combined4@assays$RNA@meta.features <- x



In [ ]:
micro.list <- SplitObject(micro.combined4, split.by = "Library")

micro.list <- lapply(X = micro.list, FUN = SCTransform)
features <- SelectIntegrationFeatures(object.list = micro.list, nfeatures = 3000)
micro.list <- PrepSCTIntegration(object.list = micro.list, anchor.features = features)
anchors <- FindIntegrationAnchors(object.list = micro.list, normalization.method = "SCT",
    anchor.features = features)
combined.sct <- IntegrateData(anchorset = anchors, normalization.method = "SCT")


In [ ]:
combined.sct <- RunPCA(combined.sct, verbose = FALSE)

In [ ]:
DefaultAssay(combined.sct) <- "integrated"

combined.sct <- RunUMAP(combined.sct, reduction = "pca", dims = 1:30)

In [ ]:
micro.combined4 = combined.sct

DefaultAssay(micro.combined4) <- "integrated"
micro.combined4 <- FindNeighbors(micro.combined4, reduction = "pca", dims = 1:30,k.param = 100, nn.method = "annoy", annoy.metric = "cosine")
micro.combined4 <- FindClusters(micro.combined4, resolution = 0.25, n.iter=100)


In [ ]:
options(repr.plot.width=12, repr.plot.height=10)

DimPlot(micro.combined4, reduction = "umap",  label = T, pt.size=1, label.size=8,raster=T) + 
    theme_classic(base_size=28)+ 
    guides(color=guide_legend(title="Cluster", override.aes = list(size=6)))



In [ ]:
options(repr.plot.width=18, repr.plot.height=4)
Idents(micro.combined4) = micro.combined4@meta.data$hash.ID
DimPlot(micro.combined4, reduction = "umap", split.by = c("Library"))

In [ ]:
# micro.combined4@meta.data$Treatment <- ifelse(micro.combined4@meta.data$hash.ID %in% c(
#     "AB01",
# "AB02",
# "AB05",
# "AB13",
# "AB14",
# "AB40",
# "AB57",
# "AB41",
# "AB49",
# "AB45",
# "AB92",
# "AB93",
# "AB88"), "Lec", "iGg1")



In [ ]:
options(repr.plot.width=10, repr.plot.height=8)

DimPlot(micro.combined4, reduction = "umap",  label = F, pt.size=1.5, label.size=8,raster=T, group.by="Treatment") + 
    theme_classic(base_size=20)+ 
    guides(color=guide_legend(title="Treatment", override.aes = list(size=6)))



In [ ]:
options(repr.plot.width=18, repr.plot.height=12)
DimPlot(micro.combined4, reduction = "umap",  label = T, pt.size=3, label.size=8,raster=T, split.by = "Library", ncol = 3) + 
    theme_classic(base_size=20)+ 
    guides(color=guide_legend(title="Cluster", override.aes = list(size=6)))


In [ ]:
DimPlot(micro.combined4, reduction = "umap",  label = T, pt.size=1, label.size=10, split.by = 'hash.ID', ncol= 5) 

In [ ]:
# Define cluster mapping
cluster_mapping <- c(
  "0" = "HM",
  "1" = "RM",
  "2" = "CRM",
  "3" = "DAM",
  "4" = "tCRM",
  "5" = "IRM",
  "6" = "HLA",
  "7" = "C3-high"
)

# Assign cluster labels
micro.combined4@meta.data$Cluster_coarse <- recode(
  as.character(micro.combined4@meta.data$integrated_snn_res.0.25), 
  !!!cluster_mapping, .default = "Prolif"
)


In [ ]:
# refactor 

micro.combined4@meta.data$Cluster_coarse  = factor(micro.combined4@meta.data$Cluster_coarse , 
                                                    levels = c("HM", "RM", "tCRM", 
                                                                "CRM","DAM", "IRM", 
                                                                "HLA","C3-high", "Prolif"))

In [ ]:
options(repr.plot.width=11, repr.plot.height=9)

DimPlot(micro.combined4, reduction = "umap",  label = T, pt.size=1.5, 
        label.size=8,raster=T, group.by = "Cluster_coarse") + 
    theme_void(base_size=26)+ 
    guides(color=guide_legend(title="Cluster", override.aes = list(size=5))) + ggtitle("")



In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
library(nebulosa)

#calculate and plot densities

micro.combined4@meta.data$Lec <- ifelse(micro.combined4@meta.data$Treatment == "Lec", 1, 0)

plot_density(micro.combined4, "Lec", size=3, pal = 'civid') + ggtitle("Lec")+ theme_void(base_size=19) + 
                     theme( plot.title = element_text(color="black", size=18, face="bold",  hjust = 0.5), 
                          legend.position = "none")  

In [ ]:
micro.combined4@meta.data$iGg1 <- ifelse(micro.combined4@meta.data$Treatment
                                        == "iGg1", 
                                        1/nrow(subset(micro.combined4@meta.data, Treatment == "iGg1")),
                                       0)

plot_density(micro.combined4, "iGg1", size=3, pal = 'civid') + ggtitle("iGg1")+ theme_void(base_size=19) + 
                     theme( plot.title = element_text(color="black", size=18, face="bold",  hjust = 0.5), 
                          legend.position = "none")  

In [ ]:
DefaultAssay(micro.combined4) <- 'SCT'

Idents(micro.combined4) <- micro.combined4@meta.data$Cluster_coarse

micro.combined4 = PrepSCTFindMarkers(micro.combined4, assay = "SCT", verbose = TRUE)
se.big.markers4_SCT <- FindAllMarkers(micro.combined4, logfc.threshold = 0.2, min.pct = 0.2, assay = "SCT", only.pos = TRUE)


In [ ]:
se.big.markers.sig <- subset(se.big.markers4_SCT, p_val_adj < 0.05)

In [ ]:
library(dittoSeq)
library(viridis)


options(repr.plot.width=29, repr.plot.height=9, repr.res = 300)
DefaultAssay(micro.combined4) <- 'RNA'

se.big.markers.top6 <-se.big.markers.sig %>%
    group_by(cluster) %>%
    slice_max(n = 6, order_by = -p_val_adj) %>%
    slice_max(n = 6, order_by = avg_log2FC)



DotPlot(micro.combined4, features = unique(se.big.markers.top6$gene), 
        group.by = "Cluster_coarse", 
       col.min =-1, col.max = 2.2, dot.min = 0.05) + 
  geom_point(aes(size=pct.exp), shape = 21) +
  scale_colour_viridis(option="inferno", direction = -1) +
  guides(size=guide_legend(override.aes=list(shape=21 , fill="white")), 
        fill = guide_legend(title="Avg. Expr.") )+ 
  scale_size(range = c(1,10)) +
  theme_classic(base_size=25) +theme(
      axis.text.x = element_text(size = 23, angle = 90, vjust = 0.7, hjust=0.95),
      axis.text.y = element_text(size = 30)) + ylab("") + xlab("") +
  labs(color = "legend title") + 
    geom_vline(xintercept = 6.5, linetype = 'dashed', linewidth = 0.3) + 
    geom_vline(xintercept = 12.5, linetype = 'dashed', linewidth = 0.3) + 
    geom_vline(xintercept = 18.5, linetype = 'dashed', linewidth = 0.3) + 
    geom_vline(xintercept = 23.5, linetype = 'dashed', linewidth = 0.3) + 
    geom_vline(xintercept = 29.5, linetype = 'dashed', linewidth = 0.3) +
    geom_vline(xintercept = 35.5, linetype = 'dashed', linewidth = 0.3) +
    geom_vline(xintercept = 40.5, linetype = 'dashed', linewidth = 0.3) +
    geom_vline(xintercept = 46.5, linetype = 'dashed', linewidth = 0.3) 


#dev.off()

In [ ]:
library(Seurat)
library(RColorBrewer)
library(gridExtra)
library(ggplot2)

# Set default assay
DefaultAssay(micro.combined4) <- 'SCT'

# Add module scores
for (name in names(micro_states)) {
  micro.combined4 <- AddModuleScore(micro.combined4, 
                                    features = list(micro_states[[name]]), 
                                    name = paste0(name, "_enriched"))
}

# Define signatures
signatures <- paste0(names(micro_states), "_enriched1")


# Generate feature plots
options(repr.plot.width = 18.5, repr.plot.height = 15)

plotlist <- lapply(signatures, function(sig) {
  FeaturePlot(micro.combined4,
              features = sig, min.cutoff = 'q1', max.cutoff = 'q99', reduction = "umap", 
              label = FALSE, pt.size = 0.7, repel = TRUE) +
    scale_colour_gradientn(colours = rev(brewer.pal(n = 11, name = "Spectral"))) +  
    theme_void(base_size = 20) + 
    theme(plot.title = element_text(hjust = 0.5, size = 20, face = "bold")) + 
    ggtitle(paste("Mancuso2024", sig, sep = "_"))
})

# Arrange plots dynamically in a grid
ncol <- 3  
nrow <- ceiling(length(plotlist) / ncol)
grid.arrange(grobs = plotlist, ncol = ncol, nrow = nrow)


In [ ]:
saveRDS(micro.combined4, 'scrna_lecMicro_cleaned_integrated_sctransform.RDS')

# Differential Expression Between Conditions and Analysis

In [ ]:
Idents(micro.combined4) <- micro.combined4@meta.data$Treatment
DefaultAssay(micro.combined4) <- 'SCT'

micro.combined4 = PrepSCTFindMarkers(micro.combined4, assay = "SCT", verbose = TRUE)
Lec_iGg1 = FindMarkers(micro.combined4, 'Lec', 'iGg1',  min.pct = 0.01, logfc.threshold = 0.005, assay = "SCT")



In [ ]:
options(repr.plot.width=18, repr.plot.height=10)
library(ggrepel)


volPlot <- function(markers, colors) {
    markers$X = rownames(markers)
    

    markers$X <- rownames(markers)
    return(ggplot(markers, aes(x=avg_log2FC, y = -log10(p_val_adj),  
                              label=ifelse(p_val_adj < 0.05 & abs(avg_log2FC) > 0.075,
         X,
         ""
       ),  fill = factor(ifelse(
                ((avg_log2FC < 0 & p_val_adj < 0.05))  ,
                "Down-regulated [166]", ifelse((avg_log2FC > 0 & p_val_adj < 0.05),
                    "Up-regulated [126]", "NS"
         )))), key_glyph = draw_key_point, color="darkgrey")+  
           
           scale_fill_manual(name = "Direction (padj < 0.05)",
                     values = colors) + 
      
           geom_point(size=4, shape=21, key_glyph = draw_key_point, stroke=0.1, color="grey2") +
 
            geom_hline(yintercept = 1.2, color = "red", linetype="dashed") +
      
           geom_vline(xintercept = 0, color = "black", linetype="dashed") +
    
           theme_bw(base_size=23) + 
          
           geom_label_repel(size=5, max.overlaps=30, fill=NA, force = 0.4, nudge_y = -1)+
          
              scale_color_manual(name = "Gene Set", values=c("red", "blue", NA,  NA))+
            scale_alpha_manual(values=c(1,1,0,0), guide='none'))+

    
            xlab("\n avg_log2FoldChange") + ylab("-log10(padj) \n") +
            guides(col = guide_legend(order = 1),fill = guide_legend(order = 2))
    

}

cbPalette <- c("#56B4E9", "grey", "#E69F00","#F0E442", "#0072B2", "#D55E00", "#CC79A7")


In [ ]:
options(repr.plot.width=18, repr.plot.height=10)

volPlot(Lec_iGg1, c("black", "grey","red"))  + ggtitle("Lec vs. IgG1")

In [ ]:
library(clusterProfiler)

df = Lec_iGg1
    df$X = rownames(df)
    original_gene_list <- df$avg_log2FC
    names(original_gene_list) <- df$X
    gene_list<-na.omit(original_gene_list)
    gene_list = sort(gene_list, decreasing = TRUE)

# Convert gene IDs 
ids<-bitr(names(original_gene_list), fromType = "SYMBOL", toType = "ENTREZID", OrgDb=org.Hs.eg.db)
dedup_ids = ids[!duplicated(ids[c("SYMBOL")]),]

df2 = df[df$X %in% dedup_ids$SYMBOL,]
df2$Y = dedup_ids$ENTREZID

# Create a vector of the gene unuiverse
gene_list <- df2$avg_log2FC
# Name vector with ENTREZ ids
names(gene_list) <- df2$Y

# omit any NA values 
gene_list<-na.omit(gene_list)

# sort the list in decreasing order (required for clusterProfiler)
gene_list = sort(gene_list, decreasing = TRUE)

In [ ]:
library(ReactomePA)

y <- gsePathway(kegg_gene_list, 
                pvalueCutoff = 0.1,
                pAdjustMethod = "BH", 
                verbose = FALSE)
head(y,)

In [ ]:
library(enrichplot)

colour <- c("\\#1F78B4", "\\#E31A1C", "\\#A9A9A9")
      colour <- stringr::str_remove_all(colour, ".*#") %>% paste0("#", .)
res = y@result

res$Treatment = ifelse(res$NES > 0 & res$p.adjust < 0.05, "Lec", 
                   ifelse(res$NES < 0 & res$p.adjust < 0.05, "IgG1",
                         "C"))

options(repr.plot.width=22, repr.plot.height=16)
ggplot(head(res, 46), aes(x = reorder(Description, -NES), y = NES, fill = Treatment)) +
      geom_bar(stat = "identity", width = 0.8) +
      scale_fill_manual(values = c("Lec" = "Red", "IgG1" = "Black", "C" = colour[3])) +
      scale_x_discrete(expand = expansion(add = .5)) +
      scale_y_continuous(breaks = seq(
        floor(min(res$NES)), ceiling(max(res$NES)),
        ceiling((ceiling(max(res$NES)) - floor(min(res$NES))) / 6)
      )) +
      coord_flip() + theme_bw(base_size = 20) + ggtitle("GSEA Lec vs. IgG1: Reactome Pathways") + xlab("")


## MASC analysis for cell states

In [ ]:
library(lme4)


In [ ]:
test.df = micro.combined4@meta.data[c('hash.ID', 'Cluster_coarse', 'Treatment')]
test.df$Cluster_coarse = as.character(test.df$Cluster_coarse )
test.df$Cluster_coarse = ifelse(test.df$Cluster_coarse == "C3-high", "C3high", test.df$Cluster_coarse)
test.df$status = as.factor(test.df$Treatment)


In [ ]:
res_masc = MASC(data = test.df, cluster = test.df$Cluster_coarse, contrast = "status", random_effects = "hash.ID")
res_masc$p.adjusted <- p.adjust(res_masc$model.pvalue, method = "fdr")


In [ ]:
res_masc[c('model.pvalue', 'statusLec.OR', 'p.adjusted')]

In [ ]:
res_masc$OR_log = log(res_masc$statusLec.OR, base = 2)
res_masc$lower_CI = log(res_masc$statusLec.OR.95pct.ci.lower, base = 2)
res_masc$higher_CI = log(res_masc$statusLec.OR.95pct.ci.upper, base = 2)

In [ ]:
res_masc$cluster = rownames(res_masc)
res_masc$cluster = gsub("cluster", "", res_masc$cluster)

In [ ]:
res_masc$cluster  = factor(res_masc$cluster, levels = c("HM","RM","tCRM", "CRM",
                                                        "DAM","IRM","HLA","C3high", "Prolif"))